In [1]:
# ============================================================
# ASTROLOGY EVENT GENERATOR - FULL VERSION
# Reads raw zodiac positions from Excel, converts to longitudes,
# computes astrological events, outputs binary event columns.
# ============================================================

import pandas as pd
import numpy as np
import re

# ------------------------------------------------------------
# 1. Zodiac Parsing Utilities
# ------------------------------------------------------------

ZODIAC_SIGNS = {
    "ARI": 0, "TAU": 30, "GEM": 60, "CAN": 90,
    "LEO": 120, "VIR": 150, "LIB": 180, "SCO": 210,
    "SAG": 240, "CAP": 270, "AQU": 300, "PIS": 330
}

def parse_zodiac_position(pos_str):
    """
    Convert 'CAP 10°34’' → absolute longitude (0–360°).
    """
    if not isinstance(pos_str, str):
        return np.nan
    
    match = re.match(r"([A-Z]{3})\s+(\d+)°(\d+)", pos_str)
    if not match:
        return np.nan
    
    sign, deg, minute = match.groups()
    base = ZODIAC_SIGNS.get(sign, 0)
    deg = float(deg)
    minute = float(minute) / 60.0
    
    return base + deg + minute


# ------------------------------------------------------------
# 2. Convert All Planet Columns to Longitudes
# ------------------------------------------------------------

PLANET_COLS = ["SU","MO","ME","VE","MA","JU","SA","UR","NE","PL","NO","LI","CH"]

def convert_all_longitudes(df):
    for col in PLANET_COLS:
        df[col + "_lon"] = df[col].apply(parse_zodiac_position)
    return df


# ------------------------------------------------------------
# 3. Astrological Event Functions (Binary Outputs)
# ------------------------------------------------------------

def event_full_moon(df, orb=6):
    angle = abs((df["MO_lon"] - df["SU_lon"]) % 360 - 180)
    return (angle < orb).astype(int)

def event_new_moon(df, orb=6):
    angle = abs((df["MO_lon"] - df["SU_lon"]) % 360)
    return (angle < orb).astype(int)

def event_retrograde(df, planet="ME"):
    """
    Retrograde if today's longitude < yesterday's longitude.
    """
    return (df[f"{planet}_lon"].diff() < 0).astype(int)

def event_planet_in_sign(df, planet="MA", sign="ARI"):
    base = ZODIAC_SIGNS[sign]
    return ((df[f"{planet}_lon"] >= base) & 
            (df[f"{planet}_lon"] < base + 30)).astype(int)

def event_aspect(df, p1="VE", p2="SA", aspect=90, orb=2):
    angle = abs((df[f"{p1}_lon"] - df[f"{p2}_lon"]) % 360 - aspect)
    return (angle < orb).astype(int)


# ------------------------------------------------------------
# 4. Generate All Events
# ------------------------------------------------------------

def generate_all_events(df):
    df["FullMoon"] = event_full_moon(df)
    df["NewMoon"] = event_new_moon(df)
    
    # Retrogrades
    df["MercuryRx"] = event_retrograde(df, "ME")
    df["VenusRx"] = event_retrograde(df, "VE")
    df["MarsRx"] = event_retrograde(df, "MA")
    
    # Sign positions
    df["MarsInAries"] = event_planet_in_sign(df, "MA", "ARI")
    df["VenusInCapricorn"] = event_planet_in_sign(df, "VE", "CAP")
    
    # Major aspects
    df["VenusSquareSaturn"] = event_aspect(df, "VE", "SA", 90, orb=2)
    df["SunOppMoon"] = event_aspect(df, "SU", "MO", 180, orb=6)
    df["SunConjMercury"] = event_aspect(df, "SU", "ME", 0, orb=4)
    df["JupiterTrineNeptune"] = event_aspect(df, "JU", "NE", 120, orb=3)
    
    return df


# ------------------------------------------------------------
# 5. MAIN PIPELINE
# ------------------------------------------------------------

input_path = r"C:\Users\oliva\OneDrive\Documents\Excel doc\astro2024.xlsx"
output_path = r"C:\Users\oliva\OneDrive\Documents\Excel doc\astro_events_output.xlsx"

df = pd.read_excel(input_path)

# Convert zodiac positions → longitudes
df = convert_all_longitudes(df)

# Generate binary astrological events
df = generate_all_events(df)

# Save output
df.to_excel(output_path, index=False)

print("Astrological event generation complete.")
print(f"Saved to: {output_path}")

Astrological event generation complete.
Saved to: C:\Users\oliva\OneDrive\Documents\Excel doc\astro_events_output.xlsx


In [3]:
# ============================================================
# ASTROLOGY EVENT GENERATOR - FULL VERSION
# Reads raw zodiac positions from Excel, converts to longitudes,
# computes astrological events, outputs binary event columns.
# ============================================================

import pandas as pd
import numpy as np
import re

# ------------------------------------------------------------
# 1. Zodiac Parsing Utilities
# ------------------------------------------------------------

ZODIAC_SIGNS = {
    "ARI": 0, "TAU": 30, "GEM": 60, "CAN": 90,
    "LEO": 120, "VIR": 150, "LIB": 180, "SCO": 210,
    "SAG": 240, "CAP": 270, "AQU": 300, "PIS": 330
}

def parse_zodiac_position(pos_str):
    """
    Convert 'CAP 10°34’' → absolute longitude (0–360°).
    """
    if not isinstance(pos_str, str):
        return np.nan
    
    match = re.match(r"([A-Z]{3})\s+(\d+)°(\d+)", pos_str)
    if not match:
        return np.nan
    
    sign, deg, minute = match.groups()
    base = ZODIAC_SIGNS.get(sign, 0)
    deg = float(deg)
    minute = float(minute) / 60.0
    
    return base + deg + minute


# ------------------------------------------------------------
# 2. Convert All Planet Columns to Longitudes
# ------------------------------------------------------------

PLANET_COLS = ["SU","MO","ME","VE","MA","JU","SA","UR","NE","PL","NO","LI","CH"]

def convert_all_longitudes(df):
    for col in PLANET_COLS:
        df[col + "_lon"] = df[col].apply(parse_zodiac_position)
    return df


# ------------------------------------------------------------
# 3. Astrological Event Functions (Binary Outputs)
# ------------------------------------------------------------

def event_full_moon(df, orb=6):
    angle = abs((df["MO_lon"] - df["SU_lon"]) % 360 - 180)
    return (angle < orb).astype(int)

def event_new_moon(df, orb=6):
    angle = abs((df["MO_lon"] - df["SU_lon"]) % 360)
    return (angle < orb).astype(int)

def event_retrograde(df, planet="ME"):
    return (df[f"{planet}_lon"].diff() < 0).astype(int)

def event_planet_in_sign(df, planet="MA", sign="ARI"):
    base = ZODIAC_SIGNS[sign]
    return ((df[f"{planet}_lon"] >= base) &
            (df[f"{planet}_lon"] < base + 30)).astype(int)

def event_aspect(df, p1="VE", p2="SA", aspect=90, orb=2):
    angle = abs((df[f"{p1}_lon"] - df[f"{p2}_lon"]) % 360 - aspect)
    return (angle < orb).astype(int)


# ------------------------------------------------------------
# NEW: Lunar Eclipse Event
# ------------------------------------------------------------

def event_lunar_eclipse(df, node_orb=12):
    """
    Lunar Eclipse = Full Moon + Moon near Node.
    node_orb defines how close the Moon must be to the Node (in degrees).
    """
    node_dist = abs((df["MO_lon"] - df["NO_lon"]) % 360)
    node_dist = np.minimum(node_dist, 360 - node_dist)  # shortest angular distance
    
    return ((df["FullMoon"] == 1) & (node_dist < node_orb)).astype(int)


# ------------------------------------------------------------
# 4. Generate All Events
# ------------------------------------------------------------

def generate_all_events(df):
    df["FullMoon"] = event_full_moon(df)
    df["NewMoon"] = event_new_moon(df)
    
    # Retrogrades
    df["MercuryRx"] = event_retrograde(df, "ME")
    df["VenusRx"] = event_retrograde(df, "VE")
    df["MarsRx"] = event_retrograde(df, "MA")
    
    # Sign positions
    df["MarsInAries"] = event_planet_in_sign(df, "MA", "ARI")
    df["VenusInCapricorn"] = event_planet_in_sign(df, "VE", "CAP")
    
    # Major aspects
    df["VenusSquareSaturn"] = event_aspect(df, "VE", "SA", 90, orb=2)
    df["SunOppMoon"] = event_aspect(df, "SU", "MO", 180, orb=6)
    df["SunConjMercury"] = event_aspect(df, "SU", "ME", 0, orb=4)
    df["JupiterTrineNeptune"] = event_aspect(df, "JU", "NE", 120, orb=3)

    # NEW: Lunar Eclipse
    df["LunarEclipse"] = event_lunar_eclipse(df)

    return df


# ------------------------------------------------------------
# 5. MAIN PIPELINE
# ------------------------------------------------------------

input_path = r"C:\Users\oliva\OneDrive\Documents\Excel doc\astro2024.xlsx"
output_path = r"C:\Users\oliva\OneDrive\Documents\Excel doc\astro_events_output.xlsx"

df = pd.read_excel(input_path)

df = convert_all_longitudes(df)
df = generate_all_events(df)

df.to_excel(output_path, index=False)

print("Astrological event generation complete.")
print(f"Saved to: {output_path}")

Astrological event generation complete.
Saved to: C:\Users\oliva\OneDrive\Documents\Excel doc\astro_events_output.xlsx
